<a href="https://colab.research.google.com/github/Thanatipz/BSC_DPDM23/blob/main/%E0%B8%84%E0%B8%B3%E0%B8%99%E0%B8%A7%E0%B8%93SPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import scipy.stats as stats
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')  # เชื่อมต่อ Google Drive

def fit_gamma_distribution(precipitation):
    precipitation = np.array(precipitation)
    precipitation = precipitation[precipitation > 0]
    if len(precipitation) == 0:
        raise ValueError("ไม่มีข้อมูลที่มากกว่า 0 สำหรับฟิต Gamma Distribution")
    shape, loc, scale = stats.gamma.fit(precipitation, floc=0)
    return shape, scale

def gamma_cdf(x, shape, scale):
    return stats.gamma.cdf(x, shape, scale=scale)

def spi(precipitation):
    precipitation = np.array(precipitation)
    if np.any(precipitation <= 0):
        print("พบค่าที่เป็น 0 หรือค่าลบในข้อมูลฝน ใช้ Gaussian Distribution แทน")
        spi_values = stats.zscore(precipitation)
    else:
        shape, scale_param = fit_gamma_distribution(precipitation)
        cdf_vals = gamma_cdf(precipitation, shape, scale_param)
        spi_values = stats.norm.ppf(cdf_vals)
    return spi_values

def convert_yyyyddd_to_date(yyyyddd):
    yyyyddd = int(yyyyddd)
    year = yyyyddd // 1000
    day_of_year = yyyyddd % 1000
    return pd.to_datetime(f"{year}-{day_of_year}", format="%Y-%j")

def read_and_aggregate_excel(file_path, sheet_name=0):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df.columns = ["Date", "Rainfall"]
    df.dropna(subset=["Date", "Rainfall"], inplace=True)
    df["Date"] = df["Date"].apply(convert_yyyyddd_to_date)
    df.set_index("Date", inplace=True)

    periods = []
    start_date = df.index[0]
    while start_date.year < df.index[-1].year:
        end_date = pd.Timestamp(year=start_date.year + 1, month=4, day=1) - pd.DateOffset(days=1)
        if end_date > df.index[-1]:
            end_date = df.index[-1]
        rainfall_sum = df.loc[start_date:end_date, "Rainfall"].sum()
        periods.append((start_date, end_date, rainfall_sum))
        start_date = pd.Timestamp(year=start_date.year + 1, month=11, day=1)

    period_df = pd.DataFrame(periods, columns=["Start Date", "End Date", "Rainfall"])
    return period_df

file_path = "/content/drive/MyDrive/SPI/rain.xlsx"
precip_data = read_and_aggregate_excel(file_path)
spi_values = spi(precip_data["Rainfall"])

spi_df = precip_data.copy()
spi_df["SPI Value"] = spi_values
spi_df["Start Date"] = spi_df["Start Date"].dt.strftime("%Y-%m-%d")
spi_df["End Date"] = spi_df["End Date"].dt.strftime("%Y-%m-%d")

print(spi_df)

output_path = "/content/drive/MyDrive/SPI/spi_results.xlsx"
spi_df.to_excel(output_path, index=False)
print(f"บันทึกไฟล์ Excel ที่: {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
    Start Date    End Date  Rainfall  SPI Value
0   2003-11-01  2004-03-31      79.6  -0.086238
1   2004-11-01  2005-03-31      16.8  -2.563467
2   2005-11-01  2006-03-31     121.1   0.809622
3   2006-11-01  2007-03-31      46.7  -1.062357
4   2007-11-01  2008-03-31      77.7  -0.134100
5   2008-11-01  2009-03-31     137.5   1.107307
6   2009-11-01  2010-03-31      49.5  -0.963370
7   2010-11-01  2011-03-31     119.0   0.769653
8   2011-11-01  2012-03-31      73.7  -0.237509
9   2012-11-01  2013-03-31      94.2   0.258163
10  2013-11-01  2014-03-31      50.5  -0.928971
11  2014-11-01  2015-03-31     118.7   0.763906
12  2015-11-01  2016-03-31      72.5  -0.269272
13  2016-11-01  2017-03-31     143.8   1.215555
14  2017-11-01  2018-03-31     109.9   0.590922
15  2018-11-01  2019-03-31      82.6  -0.012211
16  2019-11-01  2020-03-31      57.5  -0.700593
17  202